In [1]:
# this is not ready to be used for anything

In [2]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.7.1
TFX version: 1.6.1


In [3]:
import os
os.getcwd()
# should be .../Scam Busters or whatever you called this folder

'g:\\Meine Ablage\\TechLabs\\Scam Busters'

In [4]:
# save the path of the data directory
data_root = 'data\\engineered\\'
# path of the UDF module
# the UDF module contains functions which are used by the transform and train components
module_file = 'module.py'

In [5]:
pipeline_name = "busters"

# Output directory to store artifacts generated from the pipeline.
pipeline_root = os.path.join('pipelines', pipeline_name)

# Path to a SQLite DB file to use as an MLMD storage.
metadata_path = os.path.join('.','metadata', pipeline_name, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
serving_model_dir = os.path.join('serving_model', pipeline_name)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [6]:
metadata_path

'.\\metadata\\busters\\metadata.db'

In [7]:
import datetime
import os
from typing import List

# imports for tfx and tf
from tfx.components import StatisticsGen

# csv will be replaced by parquet for better performance
from tfx.components import CsvExampleGen

import tensorflow_model_analysis as tfma
from tfx.components import ExampleValidator
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.components import Trainer
from tfx.components import Evaluator
from tfx.components import SchemaGen
from tfx.components import Pusher

from tfx.dsl.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2

from tfx.dsl.components.common import resolver
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

from tfx.types import Channel

from tfx.orchestration import metadata
from tfx.orchestration import pipeline

In [8]:
# TODO(b/137289334): rename this as simple after DAG visualization is done.
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str,
                     beam_pipeline_args: List[str] = None) -> pipeline.Pipeline:

    # staging
    # time to load the dataset

    # csv will be replaced by parquet for better performance
    example_gen = CsvExampleGen(input_base=data_root)

    # two options, unsure which one is better
    # pre-transform data to have one datapoint per row
    # or load in multiple files to clena up in the transformer
    # thoughts on pre-transform: we plan to have a dataset generator to create the single data points
    # we have to make pre-transform trackable, tfx cannot do that (as far as I know)
    # statistics generator
    stats_gen = StatisticsGen(
        examples=example_gen.outputs['examples']
        # ,name='compute-eval-stats'
        )
    # schema generation
    # it should be checked if there's an existing, curated schema
    # if there is, it should be used
    # the check will be implemented later
    schema_gen = tfx.components.SchemaGen(
        statistics=stats_gen.outputs['statistics'])
    # Performs anomaly detection based on statistics and data schema.
    validate_stats = ExampleValidator(
        statistics=stats_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema'])
    # Performs transformations and feature engineering in training and serving.
    transform = Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        module_file=module_file)
    # module file refers to user defined functions that are used by transformer & trainer
    # Uses user-provided Python function that implements a model.
    trainer = Trainer(
        module_file=module_file,
        # custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args=trainer_pb2.TrainArgs(num_steps=10000),
        eval_args=trainer_pb2.EvalArgs(num_steps=5000))
    # as far as i understand this takes the recently trained model
    # only purpose is to put the latest model into the validator

    # Get the latest blessed model for model validation.
    model_resolver = resolver.Resolver(
        strategy_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
        model=Channel(type=Model),
        model_blessing=Channel(
            type=ModelBlessing)).with_id('latest_blessed_model_resolver')
    # Uses TFMA to compute a evaluation statistics over features of a model and
    # perform quality validation of a candidate model (compared to a baseline).
    eval_config = tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='tips')],
        slicing_specs=[tfma.SlicingSpec()],
        metrics_specs=[
            tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(
                    class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.6}),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': -1e-10})))
            ])
        ])

    model_analyzer = Evaluator(
        examples=example_gen.outputs['examples'],
        model=trainer.outputs['model'],
        baseline_model=model_resolver.outputs['model'],
        # Change threshold will be ignored if there is no baseline (first run).
        eval_config=eval_config)
    # Checks whether the model passed the validation steps and pushes the model
    # to a file destination if check passed.
    pusher = Pusher(
        model=trainer.outputs['model'],
        model_blessing=model_analyzer.outputs['blessing'],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=serving_model_dir)))

    components=[
            example_gen,
            stats_gen,
            schema_gen,
            validate_stats,
            transform,
            trainer,
            model_resolver,
            model_analyzer,
            pusher,
        ]

    return pipeline.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=sqlite_metadata_connection_config(metadata_path),
        # metadata.sqlite_metadata_connection_config(metadata_path),
        beam_pipeline_args=beam_pipeline_args)

In [9]:
from tfx.v1.dsl.io import fileio
from ml_metadata.proto import metadata_store_pb2

def sqlite_metadata_connection_config(
    metadata_db_uri: str) -> metadata_store_pb2.ConnectionConfig:
    """Convenience function to create file based metadata connection config.

    Args:
        metadata_db_uri: uri to metadata db.

    Returns:
        A metadata_store_pb2.ConnectionConfig based on given metadata db uri.
    """
    fileio.makedirs(os.path.dirname(metadata_db_uri))
    connection_config = metadata_store_pb2.ConnectionConfig()
    connection_config.sqlite.filename_uri = metadata_db_uri
    connection_config.sqlite.connection_mode = (
        metadata_store_pb2.SqliteMetadataSourceConfig.READWRITE_OPENCREATE)
    return connection_config

In [10]:
sqlite_metadata_connection_config(metadata_path)

sqlite {
  filename_uri: ".\\metadata\\busters\\metadata.db"
  connection_mode: READWRITE_OPENCREATE
}

In [14]:
pipeline_test = _create_pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      data_root=data_root,
      # schema path will be used if curated schemas are available
      # the code in _create_pipeline will be adapted to accept schema paths and check if they need to be created
      # schema_path=schema_path,
      module_file=module_file,
      serving_model_dir=serving_model_dir,
      metadata_path=metadata_path)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.


In [15]:
tfx.orchestration.LocalDagRunner().run(
  pipeline_test)

INFO:absl:Generating ephemeral wheel package for 'g:\\Meine Ablage\\TechLabs\\Scam Busters\\module.py' (including modules: ['module']).
INFO:absl:User module package has hash fingerprint version 8fc706649011f203c41364ea9577cc328ce401fb285fe72408ab0ceeddf8a877.
INFO:absl:Executing: ['C:\\ProgramData\\Anaconda3\\envs\\pipeline\\python.exe', 'C:\\Users\\SNKE~1\\AppData\\Local\\Temp\\tmp86vw676s\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\SNKE~1\\AppData\\Local\\Temp\\tmp3ddzd3y2', '--dist-dir', 'C:\\Users\\SNKE~1\\AppData\\Local\\Temp\\tmpcdyhrkck']
INFO:absl:Successfully built user code wheel distribution at 'pipelines\\busters\\_wheels\\tfx_user_code_Transform-0.0+8fc706649011f203c41364ea9577cc328ce401fb285fe72408ab0ceeddf8a877-py3-none-any.whl'; target user module is 'module'.
INFO:absl:Full user module path is 'module@pipelines\\busters\\_wheels\\tfx_user_code_Transform-0.0+8fc706649011f203c41364ea9577cc328ce401fb285fe72408ab0ceeddf8a877-py3-none-any.whl'
INFO

InternalError: Error when executing query: disk I/O error query: COMMIT;